In [1]:
import matplotlib as plt
import pandas as pd
import sklearn as sk

In [2]:
from pandas import read_csv
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns

In [3]:
df = pd.read_csv("df_clean.csv",delimiter=',')

In [4]:
df = df.drop(columns=["Unnamed: 0","Average Amount/transaction/day","6_month_avg_chbk_amt"])

In [5]:
df = df.rename(columns={'Is declined':'is_declined','isForeignTransaction':'is_foreign_transaction','isHighRiskCountry':'is_high_risk_country'
                   ,'isFradulent':'is_fraudulent','Total Number of declines/day':'total_number_of_declines_per_day'})

In [6]:
df.shape

(3075, 8)

In [8]:
array = df.values
X = array[:,0:7]
y = array[:,7]

In [9]:
# Splitting the data into 60% training, 20 % Validation and 20 % Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [15]:
import imblearn
import collections


In [20]:
smt = SMOTE()

In [21]:
X_train,y_train = smt.fit_resample(X_train,y_train)

In [24]:
# Choose between different algorithms for the baseline

In [27]:
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', svm.SVC(gamma='auto')))
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

LR: 0.964038 (0.013546)
KNN: 0.815142 (0.017462)
SVM: 0.688328 (0.021801)


In [28]:
# Linear regression is the best model for the use case.